In [7]:
import pyodbc
import pandas as pd
import difflib
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [2]:
server = 'dataintegrationproject.database.windows.net' 
database = 'DieZweiFragezeichen' 
username = 'di_public' 
password = 'DI_Pub2022!'  
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [3]:
q_HDI = "SELECT * FROM HDI;"
q_AdjustedNI = "SELECT * FROM AdjustedNetIncome;"
q_Countries = "SELECT * FROM Countries;"
q_CountryGdp = "SELECT * FROM CountryGdp;"
q_DurSchoolCompulsory = "SELECT * FROM DurSchoolCompulsory;"
q_LifeExpectancy = "SELECT * FROM LifeExpectancy;"
q_LiteracyRate = "SELECT * FROM LiteracyRate;"

HDI = pd.read_sql(q_HDI, cnxn)
AdjustedNetIncome = pd.read_sql(q_AdjustedNI, cnxn)
Countries = pd.read_sql(q_Countries, cnxn)
CountryGdp = pd.read_sql(q_CountryGdp, cnxn)
DurSchoolCompulsory = pd.read_sql(q_DurSchoolCompulsory, cnxn)
LifeExpectancy = pd.read_sql(q_LifeExpectancy, cnxn)
LiteracyRate = pd.read_sql(q_LiteracyRate, cnxn)


Countries

,Country Code,Region,IncomeGroup,SpecialNotes,TableName
0,ABW,Latin America & Caribbean,High income,None,Aruba
1,AFE,None,None,"26 countries, stretching from the Red Sea in t...",Africa Eastern and Southern
2,AFG,South Asia,Low income,The reporting period for national accounts dat...,Afghanistan
3,AFW,None,None,"22 countries, stretching from the westernmost ...",Africa Western and Central
4,AGO,Sub-Saharan Africa,Lower middle income,None,Angola
...,...,...,...,...,...
260,XKX,Europe & Central Asia,Upper middle income,None,Kosovo
261,YEM,Middle East & North Africa,Low income,The World Bank systematically assesses the app...,"Yemen, Rep."
262,ZAF,Sub-Saharan Africa,Upper middle income,Fiscal year end: March 31; reporting period fo...,South Africa
263,ZMB,Sub-Saharan Africa,Lower middle income,National accounts data were rebased to reflect...,Zambia


In [4]:
HDI['Country1'] = HDI['Country'].apply(lambda x: difflib.get_close_matches(x, Countries['TableName']))
HDI['Country1'] = HDI['Country1'].str[0]
HDI

,HDI Rank,Country,1990,1991,1992,1993,1994,1995,1996,1997,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,Country1
0,169,Afghanistan,0.302,0.307,0.316,0.312,0.307,0.331,0.335,0.339,...,0.477,0.489,0.496,0.5,0.5,0.502,0.506,0.509,0.511,Afghanistan
1,69,Albania,0.65,0.631,0.615,0.618,0.624,0.637,0.646,0.645,...,0.764,0.775,0.782,0.787,0.788,0.788,0.79,0.792,0.795,Albania
2,91,Algeria,0.572,0.576,0.582,0.586,0.59,0.595,0.602,0.611,...,0.728,0.728,0.729,0.736,0.74,0.743,0.745,0.746,0.748,Algeria
3,36,Andorra,..,..,..,..,..,..,..,..,...,0.836,0.858,0.856,0.863,0.862,0.866,0.863,0.867,0.868,Andorra
4,148,Angola,..,..,..,..,..,..,..,..,...,0.533,0.544,0.555,0.565,0.572,0.578,0.582,0.582,0.581,Angola
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,113,Venezuela (Bolivarian Republic of),0.644,0.654,0.66,0.662,0.662,0.666,0.668,0.67,...,0.769,0.772,0.777,0.775,0.769,0.759,0.743,0.733,0.711,NaN
185,117,Viet Nam,0.483,0.493,0.504,0.514,0.525,0.537,0.548,0.547,...,0.671,0.676,0.681,0.683,0.688,0.693,0.696,0.7,0.704,Vietnam
186,179,Yemen,0.401,0.401,0.404,0.406,0.408,0.414,0.421,0.426,...,0.506,0.504,0.509,0.502,0.483,0.474,0.467,0.468,0.47,"Yemen, Rep."
187,146,Zambia,0.421,0.417,0.416,0.419,0.414,0.415,0.416,0.416,...,0.534,0.549,0.557,0.561,0.569,0.571,0.578,0.582,0.584,Zambia


In [5]:
HDI.to_excel(r"HDI_update.xlsx")

In [21]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=1):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1

In [28]:
HDI_fuzzy = fuzzy_merge(HDI, Countries, 'Country', 'TableName', threshold=80)

In [29]:
HDI_fuzzy.to_excel(r"HDI_fuzzy.xlsx") 